In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchmetrics
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group, get_rank, get_world_size
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from rich.progress import track
from math import *
import argparse
import os

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchmetrics
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group, get_rank, get_world_size
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from rich.progress import track
from math import *
import argparse
import os

In [ ]:
df = pd.read_csv('C://Users//HP//Documents//uber.csv')
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [ ]:
df.describe()

,Unnamed: 0,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+05,200000.000000,200000.000000,200000.000000,199999.000000,199999.000000,200000.000000
mean,2.771250e+07,11.359955,-72.527638,39.935885,-72.525292,39.923890,1.684535
std,1.601382e+07,9.901776,11.437787,7.720539,13.117408,6.794829,1.385997
min,1.000000e+00,-52.000000,-1340.648410,-74.015515,-3356.666300,-881.985513,0.000000
25%,1.382535e+07,6.000000,-73.992065,40.734796,-73.991407,40.733823,1.000000
50%,2.774550e+07,8.500000,-73.981823,40.752592,-73.980093,40.753042,1.000000
75%,4.155530e+07,12.500000,-73.967154,40.767158,-73.963658,40.768001,2.000000
max,5.542357e+07,499.000000,57.418457,1644.421482,1153.572603,872.697628,208.000000


In [ ]:
df.drop(['Unnamed: 0','key'], axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199999 entries, 0 to 199999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fare_amount        199999 non-null  float64
 1   pickup_datetime    199999 non-null  object 
 2   pickup_longitude   199999 non-null  float64
 3   pickup_latitude    199999 non-null  float64
 4   dropoff_longitude  199999 non-null  float64
 5   dropoff_latitude   199999 non-null  float64
 6   passenger_count    199999 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 12.2+ MB


In [ ]:
from datetime import datetime
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5


# EDA

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'seaborn'
pio.renderers.default

def plot_lat_long(lat: np.ndarray, long: np.ndarray):
    # Select 10000 random items from the array
    random_lat = np.random.choice(lat, size=10000, replace=False)
    random_long = np.random.choice(long, size=10000, replace=False)

    # Convert the result to a numpy array
    random_lat = np.array(random_lat)
    random_long = np.array(random_long)

    # Calculate the bounding box of your points
    min_lat, max_lat = np.min(random_lat), np.max(random_lat)
    min_long, max_long = np.min(random_long), np.max(random_long)

    # Center the map on the bounding box
    center_lat = (min_lat + max_lat) / 2
    center_long = (min_long + max_long) / 2
    
    # Create a scatter plot
    fig = go.Figure(data=go.Scattergeo(
        lat=random_lat,
        lon=random_long,
        mode='markers',
        marker=dict(size=5, color='rgb(255, 0, 0)', symbol='circle', line=dict(width=0.5, color='white'))
    ))
    
    # Set mapbox style
    fig.update_layout(
        geo=dict(
            scope='world',
            projection_type='natural earth',
            showland=True,
            landcolor='lightgray',
            showlakes=True,
            lakecolor='lightblue',
            showocean=True,
            oceancolor='lightblue',
            # center=dict(lat=center_lat, lon=center_long),
        ),
        title_text="Scatter Plot of Latitude and Longitude",  # Add a title
        title_font=dict(size=20),  # Adjust title font size
        width=1100,
        height=800,
        legend=dict(
            title="Legend",  # Add a legend title
            x=0.8,  # Adjust legend position
            y=0.9  
        ),
    )
    
    # Show the plot
    return fig.show()


In [ ]:
import plotly.io as pio
pio.renderers.default = 'browser'

# Now call your function
plot_lat_long(lat=df['pickup_latitude'], long=df['pickup_longitude'])

In [ ]:
# Preparing train/val/test sets
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5


In [ ]:
df['hour'] = df['pickup_datetime'].dt.hour
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek
df['month'] = df['pickup_datetime'].dt.month

# Calculate distance
# df['distance'] = np.sqrt(
#     (df['dropoff_longitude'] - df['pickup_longitude'])**2 +
#     (df['dropoff_latitude'] - df['pickup_latitude'])**2)
def distance_transform(longitude1, latitude1, longitude2, latitude2):
    Distance = []
    
    for pos in range(len(longitude1)):
        long1,lati1,long2,lati2 = map(radians,[longitude1[pos],latitude1[pos],longitude2[pos],latitude2[pos]])
        dist_long = long2 - long1
        dist_lati = lati2 - lati1
        a = sin(dist_lati/2)**2 + cos(lati1) * cos(lati2) * sin(dist_long/2)**2
        c = 2 * asin(sqrt(a))*6371
        Distance.append(c)
       
    return Distance
df['distance_km'] = distance_transform(df['pickup_longitude'].to_numpy(),
                                                df['pickup_latitude'].to_numpy(),
                                                df['dropoff_longitude'].to_numpy(),
                                                df['dropoff_latitude'].to_numpy()
                                              )

# Time-based features
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 5)).astype(int)

df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day_of_week,month,distance_km,is_weekend,is_night
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,19,3,5,1.683323,0,0
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,20,4,7,2.457590,0,0
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,21,0,8,5.036377,0,0
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,8,4,6,1.661683,0,0
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,17,3,8,4.475450,0,0


In [ ]:
df.is_weekend.value_counts(),df.is_night.value_counts()

(is_weekend
 0    143307
 1     56692
 Name: count, dtype: int64,
 is_night
 0    153410
 1     46589
 Name: count, dtype: int64)

In [ ]:
locations=['pickup_latitude','pickup_longitude','dropoff_longitude','dropoff_latitude','fare_amount','distance_km']
for i in locations:
    lower_limit=df[i].quantile(0.02)
    upper_limit=df[i].quantile(0.99)
    df=df[(df[i]>=lower_limit) & (df[i]<=upper_limit)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167398 entries, 0 to 199999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   fare_amount        167398 non-null  float64            
 1   pickup_datetime    167398 non-null  datetime64[ns, UTC]
 2   pickup_longitude   167398 non-null  float64            
 3   pickup_latitude    167398 non-null  float64            
 4   dropoff_longitude  167398 non-null  float64            
 5   dropoff_latitude   167398 non-null  float64            
 6   passenger_count    167398 non-null  int64              
 7   hour               167398 non-null  int32              
 8   day_of_week        167398 non-null  int32              
 9   month              167398 non-null  int32              
 10  distance_km        167398 non-null  float64            
 11  is_weekend         167398 non-null  int64              
 12  is_night           167398 non-null 

In [ ]:
# df['pickup_datetime'] = df['pickup_datetime'].astype('float64')
# df['passenger_count'] = df['passenger_count'].astype('float64')
df = df[['passenger_count', 'hour', 'day_of_week', 'month', 'distance_km', 'is_weekend', 'is_night', 'fare_amount']].astype('float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167398 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   passenger_count  167398 non-null  float32
 1   hour             167398 non-null  float32
 2   day_of_week      167398 non-null  float32
 3   month            167398 non-null  float32
 4   distance_km      167398 non-null  float32
 5   is_weekend       167398 non-null  float32
 6   is_night         167398 non-null  float32
 7   fare_amount      167398 non-null  float32
dtypes: float32(8)
memory usage: 6.4 MB


In [ ]:
df.head()

,passenger_count,hour,day_of_week,month,distance_km,is_weekend,is_night,fare_amount
0,1.0,19.0,3.0,5.0,1.683323,0.0,0.0,7.5
1,1.0,20.0,4.0,7.0,2.457590,0.0,0.0,7.7
2,1.0,21.0,0.0,8.0,5.036377,0.0,0.0,12.9
3,3.0,8.0,4.0,6.0,1.661683,0.0,0.0,5.3
4,5.0,17.0,3.0,8.0,4.475450,0.0,0.0,16.0


In [ ]:
X = df.drop('fare_amount', axis=1)
y = df['fare_amount']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_tst, y_val, y_tst = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_tst.shape, y_tst.shape

((117178, 7), (117178,), (25110, 7), (25110,), (25110, 7), (25110,))

In [ ]:
scaler = StandardScaler()
Xt_scaled = scaler.fit_transform(X_train)
Xv_scaled = scaler.transform(X_val)
Xts_scaled = scaler.transform(X_tst)

In [ ]:
max(y),min(y)

(39.83000183105469, 3.5)

In [ ]:
Xt_scaled_t = torch.from_numpy(Xt_scaled)
Xv_scaled_t = torch.from_numpy(Xv_scaled)
Xts_scaled_t = torch.from_numpy(Xts_scaled)

yt_tensor = torch.from_numpy(y_train.to_numpy())
yv_tensor = torch.from_numpy(y_val.to_numpy())
yts_tensor = torch.from_numpy(y_tst.to_numpy())

In [ ]:
torch.save(Xt_scaled_t, 'X_train.pt')
torch.save(yt_tensor, 'y_train.pt')
torch.save(Xv_scaled_t, 'X_val.pt')
torch.save(yv_tensor, 'y_val.pt')

## Model Training Preparation

In [ ]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

CUDA available: False
Number of GPUs: 0


## Scikit Learn Model

In [ ]:
def huber_loss(y_true: np.ndarray , y_pred: np.ndarray, delta: float=1.0):
    error = y_true - y_pred
    is_small_error = np.abs(error) <= delta
    squared_loss = np.square(error) / 2
    linear_loss = delta * (np.abs(error) - delta / 2)
    return np.mean(np.where(is_small_error, squared_loss, linear_loss))

huber_scorer = make_scorer(huber_loss, greater_is_better=False)

In [ ]:
linear_model = LinearRegression()
linear_model.fit(Xt_scaled, y_train)
print(f"val score: {linear_model.score(Xv_scaled, y_val):.4f}")
lyv_preds = linear_model.predict(Xv_scaled)
print(f"val MAE: {mean_absolute_error(np.array(y_val), lyv_preds):.4f}")
print(f"val Huber Loss: {huber_loss(np.array(y_val), lyv_preds):.4f}")
print(f"test score: {linear_model.score(Xts_scaled, y_tst):.4f}")
lyts_preds = linear_model.predict(Xts_scaled)
print(f"test MAE: {mean_absolute_error(np.array(y_tst), lyts_preds):.4f}")
print(f"test Huber Loss: {huber_loss(np.array(y_tst), lyts_preds):.4f}")


val score: 0.7450
val MAE: 1.8227
val Huber Loss: 1.3941
test score: 0.7296
test MAE: 1.8370
test Huber Loss: 1.4086


In [ ]:
tree_model = DecisionTreeRegressor()
tree_model.fit(Xt_scaled, y_train)
print(f"val score: {tree_model.score(Xv_scaled, y_val):.4f}")
lyv_preds = tree_model.predict(Xv_scaled)
print(f"val MAE: {mean_absolute_error(np.array(y_val), lyv_preds):.4f}")
print(f"val Huber Loss: {huber_loss(np.array(y_val), lyv_preds):.4f}")
print(f"test score: {tree_model.score(Xts_scaled, y_tst):.4f}")
lyts_preds = tree_model.predict(Xts_scaled)  
print(f"test MAE: {mean_absolute_error(np.array(y_tst), lyts_preds):.4f}")
print(f"test Huber Loss: {huber_loss(np.array(y_tst), lyts_preds):.4f}")

val score: 0.5023
val MAE: 2.5217
val Huber Loss: 2.0863
test score: 0.4792
test MAE: 2.5528
test Huber Loss: 2.1156


### PyTorch Training Loop

In [ ]:
%%writefile torchtrain.py

import os
# os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group, get_rank, get_world_size
from rich.progress import track
from pathlib import Path
import argparse


def ddp_setup(rank, world_size):
    """
    Args:
        rank: Unique identifier of each process
        world_size: Total number of processes
    """
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12355"
    init_process_group(backend="nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)

class Trainer:
    def __init__(
        self,
        model: nn.Module,
        train_data: torch.utils.data.DataLoader,
        val_data: torch.utils.data.DataLoader,
        criterion,
        optimizer: torch.optim.Optimizer,
        patience: int,
        gpu_id: int,
#         save_every: int,
        save_path: str,
        max_epochs: int,
        world_size: int,
        scheduler = None,
    ) -> None:
        self.gpu_id = gpu_id
#         self.model = model.to(gpu_id)
        self.train_data = train_data
        self.val_data = val_data
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.patience = patience
        self.save_path = save_path
#         self.best_val_loss = float('inf')
        self.model = DDP(model.to(gpu_id), device_ids=[gpu_id])
        self.train_losses = np.array([{f'train_losses{i}': np.array([]) for i in range(world_size)}])
        self.val_losses = np.array([{f'val_losses{i}': np.array([]) for i in range(world_size)}])

    def _run_batch(self, source, targets):
        self.model.train()
        self.optimizer.zero_grad()
        output = self.model(source)
#         print(f"Output shape: {output.shape}, Targets shape: {targets.shape}")
#         loss_fnc = nn.L1Loss().to(self.gpu_id)
#         loss = F.l1_loss(output, targets.unsqueeze(1))
        loss = self.criterion(output, targets.unsqueeze(1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(parameters=self.model.parameters(), max_norm=1.0, norm_type=2.0)
        self.optimizer.step()
#         self.scheduler.step()
        return loss.item()
    
    def _run_eval(self, epoch):
        self.model.eval()
        total_loss = 0
        self.val_data.sampler.set_epoch(epoch)
        with torch.inference_mode():
            for source, targets in track(self.val_data, description=f"Evaluating...", style='red', complete_style='cyan', finished_style='green'):
                source = source.to(self.gpu_id)
                targets = targets.to(self.gpu_id)
                output = self.model(source)
#                 print(f"Output shape: {output.shape}, Targets shape: {targets.shape}")
#                 loss_fnc = nn.L1Loss().to(self.gpu_id)
#                 loss = F.l1_loss(output, targets.unsqueeze(1))
                loss = self.criterion(output, targets.unsqueeze(1))
                total_loss += loss.item()
#         print(f"val data len: {len(self.val_data)}")
        self.model.train()
        return total_loss / len(self.val_data)

    def _run_epoch(self, epoch, total_epochs):
        b_sz = self.train_data.batch_size
        total_loss = 0
        self.train_data.sampler.set_epoch(epoch)
        for source, targets in track(self.train_data, description=f"[GPU{self.gpu_id}] Epoch {epoch+1}/{total_epochs} | Training: {b_sz}...", style='red', complete_style='cyan', finished_style='green'):
            source = source.to(self.gpu_id)
            targets = targets.to(self.gpu_id)
            loss = self._run_batch(source, targets)
#             self.scheduler.step(val_loss)
            total_loss += loss
#         print(f"train data len: {len(self.train_data)}")
        return total_loss / len(self.train_data)

    def _save_checkpoint(self, epoch):
        ckp = self.model.module.state_dict()
        PATH = f"{self.save_path}/best_model.pt"
#         if self.gpu_id == 0:
        torch.save(ckp, PATH)
        print(f"\t\tNew best model saved at {PATH} from GPU{self.gpu_id}.")
            
    def loss_metric_tensors(self, array: np.ndarray):
        all_tensors = [torch.tensor([[array[0][j][k] for k in range(len(array[0][j]))]], dtype=torch.float32) for j in array[0].keys()]
        b = torch.cat(all_tensors, dim=0)
        return b.transpose(0, 1)
    
    def gather_tensor(self, t):
        gathered_t = [torch.zeros_like(t) for _ in range(get_world_size())]
        torch.distributed.all_gather(gathered_t, t)
        return torch.cat(gathered_t, dim=0)

    def train(self, max_epochs: int):
        b_sz = self.train_data.batch_size  # b_sz = len(next(iter(self.train_data))[0])
        should_stop = torch.zeros(1).to(self.gpu_id)
        patience_count = torch.zeros(1, dtype=torch.int32).to(self.gpu_id)
        for epoch in range(max_epochs):
            train_loss = self._run_epoch(epoch, max_epochs)
            val_loss = self._run_eval(epoch)
            print(f"\t[GPU{self.gpu_id}]  Batch: {b_sz} | Train Step: {len(self.train_data)} | Val Step: {len(self.val_data)} | Loss: {train_loss:.4f} | Val_Loss: {val_loss:.4f} | Learning Rate: {self.optimizer.param_groups[0]['lr']:.6f}")
            self.scheduler.step(val_loss)
            
            # Gather losses from all GPUs
            world_size = get_world_size()
            train_losses = [torch.zeros(1).to(self.gpu_id) for _ in range(world_size)]
            val_losses = [torch.zeros(1).to(self.gpu_id) for _ in range(world_size)]
            torch.distributed.all_gather(train_losses, torch.tensor([train_loss]).to(self.gpu_id))
            torch.distributed.all_gather(val_losses, torch.tensor([val_loss]).to(self.gpu_id))
            
            # Save losses for all GPUs
            for i in range(world_size):
                self.train_losses[0][f"train_losses{i}"] = np.append(self.train_losses[0][f"train_losses{i}"], train_losses[i].item())
                self.val_losses[0][f"val_losses{i}"] = np.append(self.val_losses[0][f"val_losses{i}"], val_losses[i].item())

            val_losses_t = self.loss_metric_tensors(self.val_losses)
            
            vl_last_item = np.min(val_losses_t[-1:].squeeze().numpy())
            bval_loss = np.min(val_losses_t.numpy())
                
            # Find the best validation loss across all GPUs
#             best_val_loss = min(val_losses).item()
#             if best_val_loss < self.best_val_loss:
#                 self.best_val_loss = best_val_loss
# #                 if self.gpu_id == 0:  # Only save on the first GPU
#                 self._save_checkpoint(epoch)
            
            improved = torch.tensor([False], dtype=torch.bool).to(self.gpu_id)
            if (len(torch.where(val_losses_t==vl_last_item)[1]) == 1):
                vl_last_gpu = torch.where(val_losses_t==vl_last_item)[1].item()
                if (vl_last_item == bval_loss) and (self.gpu_id == vl_last_gpu):
                    print(f"\t\t1:[GPU{self.gpu_id}] val_loss improved to {vl_last_item:.4f}")
                    self._save_checkpoint(epoch)
                    improved = torch.tensor([True], dtype=torch.bool).to(self.gpu_id)
                    time.sleep(2)
            elif (len(torch.where(val_losses_t==vl_last_item)[1]) > 1):
                if (vm_last_item == bval_metric):
                    vl_last_gpu_min = min(torch.where(val_losses_t==vl_last_item)[1]).item()
                    if (self.gpu_id == vl_last_gpu_min):
                        print(f"\t\t2:[GPU{self.gpu_id}] val_loss: {vm_last_item:.4f}")
                        self._save_checkpoint(epoch)
                        improved = torch.tensor([True], dtype=torch.bool).to(self.gpu_id)
                        time.sleep(2)
            else:
                pass
        
            # Synchronize patience count across all GPUs
            improved_state = self.gather_tensor(improved)
        
            # Update patience count
            if (improved_state[0] and improved_state[1]) or (improved_state[0] or improved_state[1]):
                patience_count.zero_()
#                 print(f"\n[GPU{self.gpu_id}] count zero --> {patience_count}")
            else:
                patience_count += 1
#                 print(f"\n[GPU{self.gpu_id}] count increase --> {patience_count}")

            # Synchronize patience count across all GPUs
            all_patience_counts = self.gather_tensor(patience_count)
            max_patience_count = torch.max(all_patience_counts).item()
            patience_count.fill_(max_patience_count)

#             print(f"\n[GPU{self.gpu_id}] Patience Count --> {patience_count}")
            
            if max_patience_count >= self.patience:
                print(f"\n[GPU{self.gpu_id}] Patience exceeded. Early stopping...")
#                 break
                should_stop[0] = 1
    
            # Synchronize the should_stop tensor across all GPUs
            should_stop_list = [torch.zeros(1).to(self.gpu_id) for _ in range(world_size)]
            torch.distributed.all_gather(should_stop_list, should_stop)

            # If any GPU wants to stop, all GPUs should stop
            if any(stop.item() for stop in should_stop_list):
                break
        
            time.sleep(2)
        
        # Ensure all GPUs exit the training loop together
        torch.distributed.barrier()
        
        if self.gpu_id == 0:
#             print(f"Training completed.")
            
            self.train_losses[0]['epochs'], self.val_losses[0]['epochs'] = np.arange(1, len(self.train_losses[0]['train_losses0'])+1), np.arange(1, len(self.val_losses[0]['val_losses0'])+1)
            
            np.save("train_losses.npy", self.train_losses, allow_pickle=True)
            np.save("val_losses.npy", self.val_losses, allow_pickle=True)


# Custom learning rate scheduler
class CyclicLRScheduler(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, base_lr, max_lr, step_size_up=2000, step_size_down=None, mode='triangular', gamma=1., scale_fn=None, scale_mode='cycle', cycle_momentum=True, base_momentum=0.8, max_momentum=0.9, last_epoch=-1):
        if not isinstance(optimizer, torch.optim.Optimizer):
            raise TypeError('{} is not an Optimizer'.format(type(optimizer).__name__))
        self.optimizer = optimizer
        self.base_lrs = [base_lr] * len(optimizer.param_groups)
        self.max_lrs = [max_lr] * len(optimizer.param_groups)
        self.step_size_up = step_size_up
        self.step_size_down = step_size_down if step_size_down is not None else step_size_up
        self.total_size = step_size_up + self.step_size_down
        self.mode = mode
        self.gamma = gamma
        self.scale_fn = scale_fn
        self.scale_mode = scale_mode
        self.cycle_momentum = cycle_momentum
        self.base_momentums = [base_momentum] * len(optimizer.param_groups)
        self.max_momentums = [max_momentum] * len(optimizer.param_groups)
        super(CyclicLRScheduler, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        cycle = np.floor(1 + self.last_epoch / self.total_size)
        x = 1 + self.last_epoch / self.total_size - cycle
        if x <= 0.5:
            scale_factor = x * 2
        else:
            scale_factor = (1 - x) * 2

        lrs = []
        for base_lr, max_lr in zip(self.base_lrs, self.max_lrs):
            base_height = (max_lr - base_lr) * scale_factor
            lr = base_lr + base_height
            lrs.append(lr)

        return lrs

            
class CreateDataset_(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
                
                
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size: int):
        super(LinearRegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x
          
    
def load_data_objs(batch_size: int, rank: int, world_size: int, epochs: int):
    Xtrain = torch.load('X_train.pt')
    ytrain = torch.load('y_train.pt')
    Xval = torch.load('X_val.pt')
    yval = torch.load('y_val.pt')
    train_dts = CreateDataset_(Xtrain, ytrain)
    val_dts = CreateDataset_(Xval, yval)
    train_dtl = torch.utils.data.DataLoader(train_dts, batch_size=batch_size, shuffle=False, pin_memory=True, sampler=DistributedSampler(train_dts, num_replicas=world_size, rank=rank))
    val_dtl = torch.utils.data.DataLoader(val_dts, batch_size=1, shuffle=False, pin_memory=True, sampler=DistributedSampler(val_dts, num_replicas=world_size, rank=rank))
    model = LinearRegressionModel(len(Xtrain[0]))
#     criterion = nn.L1Loss()
    criterion = nn.HuberLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, weight_decay=1e-4)
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
#     scheduler = CyclicLRScheduler(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, mode='min')
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)  # requires metric to step
#     scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, epochs=epochs, steps_per_epoch=len(train_dtl), anneal_strategy='linear')

    return train_dtl, val_dtl, model, criterion, optimizer, scheduler

def model_eval(model: nn.Module):
    print(f"\n\n{'>'*10}LinearRegression Model Evaluation{'<'*10}\n")
    model.load_state_dict(torch.load("best_model/best_model.pt"))
    X_val = torch.load('X_val.pt')
    y_val = torch.load('y_val.pt')
    val_dts = CreateDataset_(X_val, y_val)
    val_dtl = torch.utils.data.DataLoader(val_dts, batch_size=1, shuffle=False, pin_memory=True,)
    total_loss = 0
    loss_func = nn.L1Loss()
    model.eval()
    with torch.inference_mode():
        for source, target in track(val_dtl, description=f"Evaluating...", style='red', complete_style='cyan', finished_style='green'):
            output = model(source)
            loss = loss_func(output, target.unsqueeze(1))
            total_loss += loss
        print(f"\nLoss --> {(total_loss / len(val_dtl)):.4f}")

def main(rank: int, world_size: int, total_epochs: int, patience: int, batch_size: int, save_path: str):
    if rank == 0:
        print(f"{'>'*10}LinearRegression Model Training{'<'*10}\n")
    ddp_setup(rank, world_size)
    train_dtl, val_dtl, model, criterion, optimizer, scheduler = load_data_objs(batch_size, rank, world_size, total_epochs)
    trainer = Trainer(model, train_dtl, val_dtl, criterion, optimizer, patience, rank, save_path, total_epochs, world_size, scheduler)
    trainer.train(total_epochs)
    destroy_process_group()
    if rank == 0:
        print(f"\n<{'='*10}Training completed & best model saved{'='*10}>\nExiting...")
        model_eval(model.to('cpu'))
        print(f"\n<{'='*10}Evaluation completed{'='*10}>")


if __name__ == "__main__":
    start_time = time.time()
    parser = argparse.ArgumentParser(description='simple distributed training job')
    parser.add_argument('--total_epochs', default=10, type=int, help='Total epochs to train the model (default: 10)')
    parser.add_argument('--patience', default=5, type=int, help='Patience for increasing val_loss (default: 5)')
    parser.add_argument('--batch_size', default=32, type=int, help='Input batch size on each device (default: 32)')
    parser.add_argument('--save_path', default='./checkpoints', type=str, help='Path to save the best model (default: ./checkpoints)')
    args = parser.parse_args()
    
    world_size = torch.cuda.device_count()
    MODEL_PATH = Path(args.save_path)
    MODEL_PATH.mkdir(parents=True, exist_ok=True)
    mp.spawn(main, args=(world_size, args.total_epochs, args.patience, args.batch_size, MODEL_PATH), nprocs=world_size)
    end_time = time.time()
    print(f'\nTime Elapsed {(end_time - start_time):.2f} sec')

Writing torchtrain.py


In [ ]:
!python torchtrain.py --total_epochs 25 --save_path ./best_model


Time Elapsed 0.01 sec


### Model Evaluation

In [ ]:
import os
os.path.exists('data/train_losses.npy')  # Returns True if file exists, False otherwise
tl_data = pd.DataFrame(data=np.load(r'C:\Users\HP\Documents\train_losses.npy', allow_pickle=True)[0])


In [ ]:
tl_data = pd.DataFrame(data=np.load('C:/Users/HP/Documents/train_losses.npy', allow_pickle=True)[0])
vl_data = pd.DataFrame(data=np.load('C:/Users/HP/Documents/val_losses.npy', allow_pickle=True)[0])

plot_data = pd.concat([tl_data.drop('epochs', axis=1), vl_data], axis=1)
fig1 = px.line(data_frame=plot_data, x='epochs', y=['train_losses0', 'val_losses0'], height=500, title='Train Losses')
fig1.update_xaxes(title_text='Epochs',)
fig1.update_yaxes(title_text='Losses')
fig2 = px.line(data_frame=plot_data, x='epochs', y=['train_losses1', 'val_losses1'], height=500, title='Val Losses')
fig2.update_xaxes(title_text='Epochs',)
fig2.update_yaxes(title_text='Losses')
fig1.show(), fig2.show()

(None, None)

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, input_size: int):
        super(LinearRegression, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = nn.functional.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = nn.functional.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = nn.functional.leaky_relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

In [ ]:
tst_dts = MyDataset(Xts_scaled_t, yts_tensor)
tst_dtl = torch.utils.data.DataLoader(tst_dts, batch_size=1, shuffle=False, pin_memory=True,)

In [ ]:
final_model = LinearRegression(len(Xts_scaled_t[0]))

In [ ]:
print(">>>>>Test Set Evaluation<<<<<")
total_loss = 0
final_model.eval()
with torch.inference_mode():
    for source, target in track(tst_dtl, description=f"Evaluating...", style='red', complete_style='cyan', finished_style='green'):
        output = final_model(source)
#         loss_func = nn.L1Loss()
        loss_func = nn.HuberLoss()
        loss = loss_func(output, target.unsqueeze(1))
        total_loss += loss
    print(f"\tLoss --> {total_loss / len(tst_dtl):.4f}")

>>>>>Test Set Evaluation<<<<<


Output()

# Model Comparision
Linear model Val MAE --> 1.8227 | Val Huber Loss --> 1.3941

Linear model Test MAE --> 1.8370 | Test Huber Loss --> 1.4086

Dtree model Val MAE --> 2.5368 | Val Huber Loss --> 2.1008

Dtree model Test MAE --> 2.5565 | Test Huber Loss --> 2.1196

Pytorch model Val L1Loss --> 1.8933 | Val Huber Loss --> 1.8039

Pytorch model Test L1Loss --> 1.8935 | Test Huber Loss --> 1.3941

In [ ]:
!rm -rf best_model/best_model.pt